In [1]:
import numpy as np
import pandas as pd
import pypsa  
import os 
import sys  
import datetime
from pyomo.environ import Constraint

file_name =  os.path.basename(sys.argv[0])
directory = os.path.dirname(os.path.realpath(file_name))+'/Output'
n_file = 'Output/' + [s for s in os.listdir(directory) if 'EU_Network_created' in s][0]

n1 = pypsa.Network()
n1.import_from_netcdf(n_file)

n_storage = pd.read_csv('Output/n_storage.csv',parse_dates=True,index_col=0)
nz_storage = pd.read_csv('Output/nz_storage_10h.csv',parse_dates=True,index_col=0)
nz_gen = pd.read_csv('Output/nz_gen_10h.csv',parse_dates=True,index_col=0)
nz_soc = pd.read_csv('Output/nz_soc_10h.csv',parse_dates=True,index_col=0)
Indices = np.load('Output/indices.npy')

Importing PyPSA from older version of PyPSA than current version 0.17.1.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network EU_Network_created_2020-11-02.nc has buses, carriers, generators, lines, links, loads, storage_units


In [2]:
Dispatch = {}
Storage = {}
Storage_p = {}

#initialize dictionary of buses per country
Country2Bus = {}
for i in n1.buses['country'].unique():
    Country2Bus[i] = n1.buses[n1.buses['country'] == i].index

#add country tags to generators and storage units (for convenience)
n1.generators['country'] = np.array(n1.buses.loc[n1.generators['bus'],'country'])
n1.storage_units['country'] = np.array(n1.buses.loc[n1.storage_units['bus'],'country'])

#Get the list of storage units with significant standard deviation over the course of the year
Storage_units = n1.storage_units.loc[n_storage.std(axis=0)>100000].index

Lines = pd.DataFrame(index=n1.lines.index,columns=['s_nom'],data=np.array(n1.lines.s_nom))
Ren_p = pd.DataFrame(index=n1.generators_t.p_max_pu.index, columns=n1.generators_t.p_max_pu.columns, data=np.array(n1.generators_t.p_max_pu))
nz_gen_pu = (nz_gen/n1.generators.p_nom).dropna(axis=1).clip(upper=1)
n1.storage_units['cyclic_state_of_charge'] = False

#enable emergency balancing gens
n1.generators.loc[n1.generators.index.str.contains('EmGenUp2'),'p_max_pu'] = 1
n1.generators.loc[n1.generators.index.str.contains('EmGenDown'),'p_min_pu'] = -1

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
#Rules used for extra_functionality - gathered here because they are used at different times for different cases

def balance_rule(model,c):
    Country_Gens = n1.generators[n1.generators['country'] == c]
    Country_Gens = Country_Gens[~Country_Gens.index.str.contains('EmGenUp2|EmGenDown')].index
    Country_Storage = n1.storage_units[n1.storage_units['country'] == c].index 
    Gen_p = sum([model.generator_p[i,n1.snapshots[hour]] for i in Country_Gens])
    Storage_store = sum([model.storage_p_store[i,n1.snapshots[hour]] for i in Country_Storage])
    Storage_disp = sum([model.storage_p_dispatch[i,n1.snapshots[hour]] for i in Country_Storage])
    zonal_gen = nz_gen.loc[n1.snapshots[hour],Country_Gens].sum()
    zonal_storage = nz_storage.loc[n1.snapshots[hour],Country_Storage].sum()
    return  Gen_p + Storage_disp - Storage_store >= zonal_gen + zonal_storage

def storage_rule(model,su):
    final = n_storage.loc[n1.snapshots[hour],su]
    start = n_storage.loc[n1.snapshots[hour-1],su]
    if (final-start > 0) and ('hydro' in su):
        inflow = n1.storage_units_t.inflow.loc[n1.snapshots[hour],su]
        if start + inflow > final:
            return model.state_of_charge[su,n1.snapshots[hour]] >= final
        else:
            return model.state_of_charge[su,n1.snapshots[hour]] >= start + inflow
    else:
        return model.state_of_charge[su,n1.snapshots[hour]] >= final
    
def extra_functionality(network,snapshots):
    model = network.model
    model.balance = Constraint(n1.buses['country'].unique(),rule=balance_rule)
    model.storage = Constraint(Storage_units,rule=storage_rule)

In [4]:
#initialize generator output dataframe
Dispatch = pd.DataFrame(index=n1.snapshots[Indices],columns=n1.generators.index) 
Storage = pd.DataFrame(index=n1.snapshots[Indices],columns=n1.storage_units.index) 
Storage_p = pd.DataFrame(index=n1.snapshots[Indices],columns=n1.storage_units.index)

for i in n1.buses['country'].unique():
    Country_Lines = n1.lines.loc[[(n1.lines.loc[m,'bus0'] in Country2Bus[i])|(n1.lines.loc[m,'bus1'] in Country2Bus[i]) for m in n1.lines.index]].index
    Country_Storage = n1.storage_units.loc[n1.storage_units['country'] == i].index
    Country_Gens = n1.generators[n1.generators['country'] == i].index        

    n1.lines.s_nom = np.Inf
    n1.lines.s_nom[Country_Lines] = Lines.loc[Country_Lines,'s_nom']        
    n1.storage_units_t.state_of_charge_set.drop(columns = n1.storage_units_t.state_of_charge_set.columns,inplace=True)
    n1.storage_units_t.state_of_charge_set = pd.concat([n1.storage_units_t.state_of_charge_set,nz_soc],axis=1)
    n1.storage_units_t.state_of_charge_set.drop(columns = Country_Storage,inplace=True)

    n1.generators_t.p_max_pu.drop(columns = n1.generators_t.p_max_pu.columns,inplace=True) 
    n1.generators_t.p_min_pu.drop(columns = n1.generators_t.p_min_pu.columns,inplace=True)
    n1.generators_t.p_max_pu = pd.concat([n1.generators_t.p_max_pu,nz_gen_pu],axis=1)
    n1.generators_t.p_min_pu = pd.concat([n1.generators_t.p_min_pu,nz_gen_pu],axis=1)
    n1.generators_t.p_max_pu.drop(columns = list(set(nz_gen_pu.columns) & set(Country_Gens)),inplace=True)
    n1.generators_t.p_min_pu.drop(columns = list(set(nz_gen_pu.columns) & set(Country_Gens)),inplace=True)
    n1.generators_t.p_max_pu.drop(columns = n1.generators_t.p_max_pu.columns[n1.generators_t.p_max_pu.columns.str.contains('EmGenUp2|EmGenDown')],inplace=True)
    n1.generators_t.p_min_pu.drop(columns = n1.generators_t.p_min_pu.columns[n1.generators_t.p_min_pu.columns.str.contains('EmGenUp2|EmGenDown')],inplace=True)
    n1.generators_t.p_max_pu = pd.concat([n1.generators_t.p_max_pu,Ren_p[list(set(Ren_p.columns) & set(Country_Gens))]],axis=1)


    for j in Indices:
        print(i,':',j)
        print(datetime.datetime.now())
        hour = j
        timesteps = np.array([hour])
        n1.storage_units['state_of_charge_initial'] = n_storage.iloc[hour-1]
        n1.lopf(n1.snapshots[timesteps],solver_name='gurobi',extra_functionality=extra_functionality)

    Dispatch.loc[:,Country_Gens] = (n1.generators_t.p.loc[n1.snapshots[Indices],Country_Gens]/(n1.generators.p_nom.replace(0,np.nan))).dropna(axis=1)
    Storage.loc[:,Country_Storage] = n1.storage_units_t.state_of_charge.loc[n1.snapshots[Indices],Country_Storage]
    Storage_p.loc[:,Country_Storage] = n1.storage_units_t.p.loc[n1.snapshots[Indices],Country_Storage]


/home/poweruser/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


PT : 2723
2020-11-11 14:29:08.056553


INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 2611732.616877904
  Upper bound: 2611732.616877904
  Number of objectives: 1
  Number of constraints: 6508
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 65758
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 3449277.203449559
  Upper bound: 3449277.203449559
  Number of objectives: 1
  Number of constraints: 6193
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 65424
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 8030347.974536163
  Upper bound: 8030347.974536163
  Number of objectives: 1
  Number of constraints: 6193
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 65424
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `kirchhoff` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x22368_copy
  Lower bound: 822646.9001186676
  Upper bound: 822646.9001186676
  Number of objectives: 1
  Number of constraints: 6193
  Number of variables: 22368
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 22368
  Number of nonzeros: 65424
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination mes

In [6]:
Dispatch_p0 = Dispatch*n1.generators.p_nom
Dispatch_p0.to_csv('Output/n1_gen_10h.csv')
Storage_p.to_csv('Output/n1_storage_10h.csv')